# Modelo Para Previsão de Doenças Usando Registros Médicos Eletrônicos - Parte 2</font>

<h2>Primeira Versão do Modelo</h2>

Agora que temos os dados devidamente processados, limpos e transformados. É hora de aplicar técnicas de Machine Learning com o objetivo de encontrar o padrão nos dados históricos para conseguir prever o futuro com base nesses padrões.

Esta parte 2 tem um caráter mais introdutório aos dados estudados. Vamos aplicar alguns algoritmos sem muitos critérios. O objetivo é realizar uma primeira modelagem e a partir disso tentar melhorar com técnicas mais avançadas.

Irei aplicar três técnicas:
* **KNN**: Vamos verificar se a relação por proximidade de vizinhança se mostra boa o suficiente;
* **Regressão Logística**: Um método linear generalizado. A regressão logística é útil para modelar a probabilidade de um evento ocorrer em função de outros fatores.
* **Random Forest**: E também irei utilizar a técnicas de árvores de decisão.

<h3>Importações das bibliotecas</h3>

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pandas as pd

from imblearn.under_sampling import RandomUnderSampler

<h3>Carrega os Dados Pré-Processados</h3>

Vamos carregar os dados já transformados e padronizados que foram salvos no notebook anterior diretamente do bucket S3.

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

# Altere para o nome do seu bucket
s3_bucket = 'dsa-deploy-app'
prefix = 'dados'

In [ ]:
raiz = 's3://{}/{}/'.format(s3_bucket, prefix)
print(raiz)

In [ ]:
#Dados de treino
df_treino = pd.read_csv(raiz + 'treino.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_treino.head())

#Dados de teste
df_teste = pd.read_csv(raiz + 'teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_teste.head())

**Carregando os dados de maneira local**

In [2]:
#Dados de treino
df_treino = pd.read_csv('dados/treino.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_treino.head())

#Dados de teste
df_teste = pd.read_csv('dados/teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_teste.head())

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,1,-0.086157,-1.336905,1.309491,0.314870
1,0,-0.368844,-1.336905,-1.376230,0.807519
2,0,1.215651,0.735390,1.426434,-0.276827
3,0,-1.485990,-1.336905,-1.376230,-1.195959
4,0,-1.509101,-1.336905,-1.376230,-0.031728


,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,-1.207477,-0.328664,-1.128684,0.542470
1,0,1.386485,-0.633929,-0.674439,-0.614477
2,0,1.013063,-0.054860,-0.096215,-1.195959
3,0,0.084261,0.480612,-0.732890,0.046293
4,0,-1.069768,-0.723112,0.199918,-1.195959


<h4>Separando variáveis preditoras da variável target</h4>

In [3]:
X_treino = df_treino[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']].values
y_treino = df_treino['class']

X_teste = df_teste[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']]
y_teste = df_teste['class']

y_treino

0       1
1       0
2       0
3       0
4       0
       ..
7194    1
7195    0
7196    0
7197    0
7198    0
Name: class, Length: 7199, dtype: int64

<h4>Balanceando o dataset</h4>

Como o dataset está bem desbalanceado, isso pode prejudicar nas previsões do modelo, portanto, iremos aplicar técnicas para balancear o dataset com o objetivo de deixá-lo balanceado e dessa forma não prejudicar o modelo.

In [4]:
y_treino.value_counts()

0    5816
1    1383
Name: class, dtype: int64

In [5]:
rus = RandomUnderSampler()
X_res, y_res = rus.fit_resample(X_treino, y_treino)

In [6]:
y_res.value_counts()

0    1383
1    1383
Name: class, dtype: int64

---
<h2>Primeira modalagem</h2>

<h3>KNN - K Nearest Neighbors</h3>

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_res, y_res)

preds = knn.predict(X_teste)
f_score = f1_score(preds, y_teste)
ac_score = accuracy_score(preds, y_teste)

print('F1 score em Teste = {:.2f}%'.format(f_score * 100))
print('Acurácia em Teste = {:.2f}%'.format(ac_score * 100))

F1 score em Teste = 26.23%
Acurácia em Teste = 49.38%


C:\Users\krupc\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


<h3>Logistic Regression</h3>

In [8]:
logisticReg = LogisticRegression()
logisticReg.fit(X_res, y_res)

preds = logisticReg.predict(X_teste)
f_score = f1_score(preds, y_teste)
ac_score = accuracy_score(preds, y_teste)

print('F1 score em Teste = {:.2f}%'.format(f_score * 100))
print('Acurácia em Teste = {:.2f}%'.format(ac_score * 100))

F1 score em Teste = 26.22%
Acurácia em Teste = 47.25%


C:\Users\krupc\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


<h3>Random Forest</h3>

In [9]:
randomForest = RandomForestClassifier()
randomForest.fit(X_res, y_res)

preds = randomForest.predict(X_teste)
f_score = f1_score(preds, y_teste)
ac_score = accuracy_score(preds, y_teste)

print('F1 score em Teste = {:.2f}%'.format(f_score * 100))
print('Acurácia em Teste = {:.2f}%'.format(ac_score * 100))

F1 score em Teste = 25.55%
Acurácia em Teste = 49.00%


C:\Users\krupc\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


---
<h2>Considerações Finais</h2>

Vemos que a precisão dos modelos foram horríveis em todos os casos. Se não tivessimos balanceado o dataset, a acurácia teria sido de mais de 80%. Quais são as conclusões até aqui:
* Balancear o dataset foi muito importante para entender como os dados realmente se comportam;
* Esses dados estão enviesados, após todos os estudos eu chego a conclusão de que ou os dados foram simulados ou o dataset não condizem com a realidade. Baseado em que:
    1. A maioria das pessoas possuiam magreza extrema;
    2. A relação entre todas as categorias de atributos estavam proporcionalmente balanceados, e isso é bem difícil de acontecer na realidade;
    3. Mesmo balanceando o dataset, o modelo não conseguiu aprender a relação entre os dados;

---
<h2>Continua...</h2>

O objetivo nos próximos notebooks será estudar o Amazon SageMaker.
* Vamos criar e treinar um modelo no ambiente em nuvem;
* Vamos gerar um endpoint para realizar previsões a partir do modelo criado;